In [41]:
import pandas as pd
import numpy as np 
import sklearn.datasets as dta
import scipy.stats as st
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.decomposition import LatentDirichletAllocation
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, Model
import tensorflow as tf

In [42]:
path = "../CO_Accidents.csv"
df = pd.read_csv(path,index_col=0)
df.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,A-512446,Source2,3,2022-09-08 13:03:18,2022-09-08 13:32:26,39.848122,-104.985306,NaN,NaN,0.0,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,A-512565,Source2,1,2022-09-08 08:44:08,2022-09-08 09:13:40,39.776600,-105.062630,NaN,NaN,0.0,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,A-512637,Source2,2,2022-09-08 06:57:38,2022-09-08 07:27:23,39.899731,-104.868507,NaN,NaN,0.0,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,A-512643,Source2,1,2022-09-08 08:45:51,2022-09-08 09:15:28,39.618809,-104.773201,NaN,NaN,0.0,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,A-512644,Source2,3,2022-09-08 08:42:23,2022-09-08 09:11:57,39.725094,-105.012817,NaN,NaN,0.0,...,False,False,False,False,False,False,Day,Day,Day,Day


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90885 entries, 512415 to 7728193
Data columns (total 46 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     90885 non-null  object 
 1   Source                 90885 non-null  object 
 2   Severity               90885 non-null  int64  
 3   Start_Time             90885 non-null  object 
 4   End_Time               90885 non-null  object 
 5   Start_Lat              90885 non-null  float64
 6   Start_Lng              90885 non-null  float64
 7   End_Lat                47671 non-null  float64
 8   End_Lng                47671 non-null  float64
 9   Distance(mi)           90885 non-null  float64
 10  Description            90885 non-null  object 
 11  Street                 90727 non-null  object 
 12  City                   90877 non-null  object 
 13  County                 90885 non-null  object 
 14  State                  90885 non-null  object 
 15  

In [44]:
df.describe()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,90885.000000,90885.000000,90885.000000,47671.000000,47671.000000,90885.000000,89602.000000,73624.000000,89518.000000,89805.000000,89255.000000,84678.000000,62826.000000
mean,2.443902,39.537114,-105.088422,39.496544,-105.190225,0.908223,47.536737,41.517384,52.383889,25.790389,11.280650,8.511995,0.001544
std,0.658282,0.572171,0.727363,0.664135,0.911719,2.930441,22.323277,26.108202,26.653627,2.962336,9.507933,6.273117,0.014303
min,1.000000,36.993870,-109.047048,36.993840,-109.283380,0.000000,-89.000000,-89.000000,1.000000,16.820000,0.000000,0.000000,0.000000
25%,2.000000,39.473172,-105.053238,39.322652,-105.181332,0.000000,30.000000,21.000000,29.000000,24.170000,10.000000,5.000000,0.000000
50%,2.000000,39.710964,-104.980141,39.696670,-104.987440,0.037000,46.900000,38.000000,50.000000,24.500000,10.000000,7.000000,0.000000
75%,3.000000,39.785191,-104.847168,39.802100,-104.860438,0.849000,64.900000,63.000000,77.000000,29.830000,10.000000,11.500000,0.000000
max,4.000000,41.002288,-102.044781,41.056590,-102.030371,254.399994,104.000000,104.000000,100.000000,30.940000,90.000000,254.300000,1.050000


In [45]:
len(df)

90885

In [46]:
df_drop = df.drop(columns = ["Source", "ID", "Start_Time", "End_Time", "Start_Lat", "Start_Lng", "Description", "Timezone", "Weather_Timestamp", "Airport_Code", "Country", "Zipcode", "Street", "City", "County", "State", "Distance(mi)", "End_Lat", "End_Lng"])
df_drop

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93.0,93.0,11.0,24.37,10.0,NE,8.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86.0,86.0,13.0,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71.0,71.0,26.0,24.62,10.0,SW,13.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78.0,78.0,21.0,24.23,10.0,SW,12.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86.0,86.0,13.0,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728172,2,86.0,86.0,14.0,24.38,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80.0,80.0,27.0,24.97,10.0,S,8.0,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728187,3,82.0,82.0,20.0,24.37,10.0,ESE,13.0,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76.0,76.0,36.0,25.19,10.0,NNE,10.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [47]:
df_drop = df_drop.drop_duplicates()
df_drop = df_drop.dropna(how='any')

In [48]:
non_numeric = df_drop['Temperature(F)'].apply(lambda x: isinstance(x, str))
non_numeric.value_counts()
df_drop.fillna(0)
df_drop['Temperature(F)'].info()

<class 'pandas.core.series.Series'>
Index: 43918 entries, 512415 to 7728193
Series name: Temperature(F)
Non-Null Count  Dtype  
--------------  -----  
43918 non-null  float64
dtypes: float64(1)
memory usage: 686.2 KB


In [49]:
df_drop['Wind_Direction'].value_counts()

Wind_Direction
CALM        5613
N           4357
S           3947
W           2938
NNW         2748
NNE         2347
SSE         2307
E           2121
SSW         2104
NW          1826
SE          1818
NE          1765
SW          1764
WSW         1751
WNW         1669
ESE         1633
ENE         1505
VAR         1424
North        180
East          40
South         34
West          18
Variable       9
Name: count, dtype: int64

In [50]:
df_drop['Wind_Direction'] = df_drop['Wind_Direction'].astype(str)
df_drop['Wind_Direction'] = df_drop['Wind_Direction'].str.upper()
df_drop['Wind_Direction'].value_counts()

Wind_Direction
CALM        5613
N           4357
S           3947
W           2938
NNW         2748
NNE         2347
SSE         2307
E           2121
SSW         2104
NW          1826
SE          1818
NE          1765
SW          1764
WSW         1751
WNW         1669
ESE         1633
ENE         1505
VAR         1424
NORTH        180
EAST          40
SOUTH         34
WEST          18
VARIABLE       9
Name: count, dtype: int64

In [51]:
#Collect non-numerics

non_num_df = df_drop.select_dtypes(include=['object', 'bool'])
print(non_num_df)


        Wind_Direction Weather_Condition  Amenity   Bump  Crossing  Give_Way  \
512415              NE     Partly Cloudy    False  False     False     False   
512525             WNW              Fair    False  False     False     False   
512588              SW              Fair    False  False      True     False   
512593              SW              Fair    False  False      True     False   
512594             WNW              Fair    False  False     False     False   
...                ...               ...      ...    ...       ...       ...   
7728170              E     Mostly Cloudy    False  False     False     False   
7728172              E     Mostly Cloudy    False  False     False     False   
7728181              S            Cloudy    False  False     False     False   
7728192            NNE     Partly Cloudy    False  False     False     False   
7728193            NNE     Partly Cloudy    False  False     False     False   

         Junction  No_Exit  Railway  Ro

In [52]:
df_drop['Temperature(F)'].fillna(0, inplace=True)
df_drop['Temperature(F)'] = np.ceil(df_drop['Temperature(F)'])
df_drop['Temperature(F)'] = df_drop['Temperature(F)'].apply(int)
#df_drop['Temperature(F)'] = round(df_drop['Temperature(F)'].apply(int))

df_drop

# df_drop['Temperature(F)'] = pd.to_numeric(df_drop['Temperature(F)'], errors='coerce')
# df_drop.dropna(subset=['Temperature(F)'], inplace=True)
# df_drop['Temperature(F)'] = df_drop['Temperature(F)'].astype(int)


,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93.0,11.0,24.37,10.0,NE,8.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86,86.0,13.0,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71,71.0,26.0,24.62,10.0,SW,13.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78,78.0,21.0,24.23,10.0,SW,12.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86,86.0,13.0,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86.0,14.0,24.38,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728172,2,86,86.0,14.0,24.38,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80,80.0,27.0,24.97,10.0,S,8.0,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76,76.0,36.0,25.19,10.0,NNE,10.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [53]:
df_drop['Wind_Chill(F)'].fillna(0, inplace=True)
df_drop['Wind_Chill(F)'] = np.ceil(df_drop['Wind_Chill(F)'])
df_drop['Wind_Chill(F)'] = df_drop['Wind_Chill(F)'].apply(int)
#df_drop['Wind_Chill(F)'] = round(df_drop['Wind_Chill(F)'].apply(int))
df_drop

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11.0,24.37,10.0,NE,8.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86,86,13.0,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71,71,26.0,24.62,10.0,SW,13.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78,78,21.0,24.23,10.0,SW,12.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86,86,13.0,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14.0,24.38,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728172,2,86,86,14.0,24.38,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80,80,27.0,24.97,10.0,S,8.0,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76,76,36.0,25.19,10.0,NNE,10.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [54]:
df_drop['Humidity(%)'].fillna(0, inplace=True)
df_drop['Humidity(%)'] = np.ceil(df_drop['Humidity(%)'])
df_drop['Humidity(%)'] = df_drop['Humidity(%)'].apply(int)
#df_drop['Humidity(%)'] = round(df_drop['Humidity(%)'].apply(int))
df_drop

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11,24.37,10.0,NE,8.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86,86,13,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71,71,26,24.62,10.0,SW,13.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78,78,21,24.23,10.0,SW,12.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86,86,13,24.43,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14,24.38,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728172,2,86,86,14,24.38,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80,80,27,24.97,10.0,S,8.0,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76,76,36,25.19,10.0,NNE,10.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [55]:
df_drop['Pressure(in)'].fillna(0, inplace=True)
df_drop['Pressure(in)'] = np.ceil(df_drop['Pressure(in)'])
df_drop['Pressure(in)'] = df_drop['Pressure(in)'].apply(int)
#df_drop['Pressure(in)'] = round(df_drop['Pressure(in)'].apply(int))
df_drop

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11,25,10.0,NE,8.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86,86,13,25,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71,71,26,25,10.0,SW,13.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78,78,21,25,10.0,SW,12.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86,86,13,25,10.0,WNW,7.0,0.0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14,25,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728172,2,86,86,14,25,10.0,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80,80,27,25,10.0,S,8.0,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76,76,36,26,10.0,NNE,10.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [56]:
df_drop['Visibility(mi)'].fillna(0, inplace=True)
df_drop['Visibility(mi)'] = np.ceil(df_drop['Visibility(mi)'])
df_drop['Visibility(mi)'] = df_drop['Visibility(mi)'].apply(int)
#df_drop['Visibility(mi)'] = round(df_drop['Visibility(mi)'].apply(int))
df_drop

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11,25,10,NE,8.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86,86,13,25,10,WNW,7.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71,71,26,25,10,SW,13.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78,78,21,25,10,SW,12.0,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86,86,13,25,10,WNW,7.0,0.0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14,25,10,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728172,2,86,86,14,25,10,E,6.0,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80,80,27,25,10,S,8.0,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76,76,36,26,10,NNE,10.0,0.0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [57]:
df_drop['Wind_Speed(mph)'].fillna(0, inplace=True)
df_drop['Wind_Speed(mph)'] = np.ceil(df_drop['Wind_Speed(mph)'])
df_drop['Wind_Speed(mph)'] = df_drop['Wind_Speed(mph)'].apply(int)
#df_drop['Wind_Speed(mph)'] = round(df_drop['Wind_Speed(mph)'].apply(int))
df_drop


,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11,25,10,NE,8,0.0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86,86,13,25,10,WNW,7,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71,71,26,25,10,SW,13,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78,78,21,25,10,SW,12,0.0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86,86,13,25,10,WNW,7,0.0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14,25,10,E,6,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728172,2,86,86,14,25,10,E,6,0.0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80,80,27,25,10,S,8,0.0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76,76,36,26,10,NNE,10,0.0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [58]:
df_drop['Precipitation(in)'].fillna(0, inplace=True)
df_drop['Precipitation(in)'] = np.ceil(df_drop['Precipitation(in)'])
df_drop['Precipitation(in)'] = df_drop['Precipitation(in)'].apply(int)
df_drop['Precipitation(in)'] = round(df_drop['Precipitation(in)'].apply(int))
df_drop

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11,25,10,NE,8,0,Partly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
512525,1,86,86,13,25,10,WNW,7,0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512588,2,71,71,26,25,10,SW,13,0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512593,1,78,78,21,25,10,SW,12,0,Fair,...,False,False,False,False,True,False,Day,Day,Day,Day
512594,3,86,86,13,25,10,WNW,7,0,Fair,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14,25,10,E,6,0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728172,2,86,86,14,25,10,E,6,0,Mostly Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728181,3,80,80,27,25,10,S,8,0,Cloudy,...,False,False,False,False,False,False,Day,Day,Day,Day
7728192,4,76,76,36,26,10,NNE,10,0,Partly Cloudy,...,False,False,False,False,False,False,Night,Night,Day,Day


In [59]:
df_drop['Wind_Direction'] = df_drop['Wind_Direction'].replace({'NNW': 'NW', 
                                                                'SSE': 'SE', 
                                                                'Calm':'CALM',
                                                                'SSW': 'SW',
                                                                'NNE': 'NE',
                                                                'WSW': 'SW',
                                                                'WNW': 'NW',
                                                                'ESE': 'SE',
                                                                'ENE': 'NE',
                                                                'S': 'SOUTH',
                                                                'N': 'NORTH',
                                                                'W': 'WEST',
                                                                'E': 'EAST'
                                                    })

# df_drop = df_drop[df_drop['Wind_Direction'] != 'VAR']
# df_drop = df_drop[df_drop['Wind_Direction'] != 'Variable']

df_drop['Wind_Direction'] = df_drop['Wind_Direction'].str.upper()

df_drop = df_drop[(df_drop['Wind_Direction'] != 'VAR') & (df_drop['Wind_Direction'] != 'Variable')]
df_drop['Wind_Direction'].value_counts()

Wind_Direction
NW          6243
SE          5758
SW          5619
NE          5617
CALM        5613
NORTH       4537
SOUTH       3981
WEST        2956
EAST        2161
VARIABLE       9
Name: count, dtype: int64

In [60]:
# Preprocess "Wind_Direction" column (one-hot encoding)
wind_encoder = OneHotEncoder(sparse_output=False)
wind_encoded = wind_encoder.fit_transform(df_drop[['Wind_Direction']])
wind_columns = wind_encoder.get_feature_names_out(['Wind_Direction'])
df_wind_encoded = pd.DataFrame(wind_encoded, columns=wind_columns)
df_wind_encoded.head()

,Wind_Direction_CALM,Wind_Direction_EAST,Wind_Direction_NE,Wind_Direction_NORTH,Wind_Direction_NW,Wind_Direction_SE,Wind_Direction_SOUTH,Wind_Direction_SW,Wind_Direction_VARIABLE,Wind_Direction_WEST
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [61]:
pd.options.display.max_rows=100
df_drop['Weather_Condition'].value_counts()

Weather_Condition
Fair                            16648
Mostly Cloudy                    7501
Partly Cloudy                    6312
Cloudy                           3762
Light Snow                       3444
Fair / Windy                      698
Light Rain                        592
Snow                              467
Mostly Cloudy / Windy             399
Fog                               333
Partly Cloudy / Windy             323
Showers in the Vicinity           157
Light Drizzle                     146
Thunder in the Vicinity           144
Haze                              142
Cloudy / Windy                    134
Smoke                             132
Light Rain with Thunder           128
Heavy Snow                        121
Wintry Mix                        113
Light Snow / Windy                109
Overcast                           84
T-Storm                            71
Thunder                            66
Rain                               66
Light Freezing Drizzle          

In [62]:
df_drop['Weather_Condition'] = df_drop['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',str(x)))
df_drop['Weather_Condition'] = df_drop['Weather_Condition'].str.replace(" ", "")

pd.options.display.max_rows=100
df_drop['Weather_Condition'].value_counts()

/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_91011/344051574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['Weather_Condition'] = df_drop['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',str(x)))
/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_91011/344051574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['Weather_Condition'] = df_drop['Weather_Condition'].str.replace(" ", "")


Weather_Condition
Fair                         16648
MostlyCloudy                  7501
PartlyCloudy                  6312
Cloudy                        3762
LightSnow                     3444
FairWindy                      698
LightRain                      592
Snow                           467
MostlyCloudyWindy              399
Fog                            333
PartlyCloudyWindy              323
ShowersintheVicinity           157
LightDrizzle                   146
ThunderintheVicinity           144
Haze                           142
CloudyWindy                    134
Smoke                          132
LightRainwithThunder           128
HeavySnow                      121
WintryMix                      113
LightSnowWindy                 109
Overcast                        84
TStorm                          71
Thunder                         66
Rain                            66
LightFreezingDrizzle            61
LightRainWindy                  41
SnowWindy                       30
He

In [63]:
# df_drop['Weather_Condition'] = df_drop['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',str(x)))
# df_drop['Weather_Condition'].str.replace(" ", "")


df_drop['Weather_Condition'] = df_drop['Weather_Condition'].replace({'Overcast': 'Cloudy', 
                                                                'LightFreezingDrizzle': 'LightDrizzle', 
                                                                'SnowWindy':'LightSnowWindy',
                                                                'LightRainWindy': 'LightSnowWindy',
                                                                'NAPrecipitation': 'Rain',
                                                                'HeavySnowWindy': 'Snow',
                                                                'HazeWindy': 'Haze',
                                                                'LightRainShower': 'Rain',
                                                                'FogWindy': 'Fog',
                                                                'TStormWindy': 'TStorm',
                                                                'PatchesofFog': 'Fog',
                                                                'HeavyRain': 'Rain',
                                                                'ThunderWindy': 'Thunder',
                                                                'BlowingDustWindy': 'CloudyWindy',
                                                                'BlowingSnowWindy': 'MostlyCloudyWindy',
                                                                'HeavyTStorm': 'TStorm',
                                                                'LightFreezingFog': 'Fog',
                                                                'HeavyTStormWindy': 'TStorm',
                                                                'PartialFog': 'Fog',
                                                                'WidespreadDustWindy': 'Fog',
                                                                'ScatteredClouds': 'Cloudy',
                                                                'DrizzleandFog': 'Fog',
                                                                'RainWindy': 'Windy',
                                                                'SquallsWindy': 'Windy',
                                                                'BlowingSnow': 'Snow',
                                                                'SmokeWindy': 'Smoke',
                                                                'LightSnowwithThunder': 'LightSnow',
                                                                'HeavyRainWindy': 'Rain',
                                                                'LightFreezingRain': 'Rain',
                                                                'Clear': 'Fair',
                                                                'BlowingDust': 'MostlyCloudyWindy',
                                                                'LightRainShowerWindy': 'LightSnowWindy',
                                                                'HeavyDrizzle': 'LightDrizzle',
                                                                'ThunderWintryMix': 'WintryMix',
                                                                'LightDrizzleWindy': 'LightDrizzle',
                                                                'LightSnowShower': 'LightSnow',
                                                                'LightThunderstormsandSnow': 'LightSnow',
                                                                'LightThunderstormsandRain': 'LightRain',
                                                                'FreezingDrizzle': 'Drizzle',
                                                                'Squalls': 'FairWindy',
                                                                'Windy': 'FairWindy',
                                                                'Thunderstorm': 'Thunder',
                                                                'ColoudyWind': 'MostlyCloudyWindy',
                                                                'Drizzle': 'LightDrizzle',
                                                                'CloudyWindy': 'MostlyCloudyWindy',
                                                                'Clouds': 'Cloudy',
                                                                'LightSnowWindy': 'FairWindy',
                                                                'LightSnowWindy': 'LightSnowWindy'
                                                    })

pd.options.display.max_rows=100
df_drop['Weather_Condition'].value_counts()


/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_91011/2184661440.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['Weather_Condition'] = df_drop['Weather_Condition'].replace({'Overcast': 'Cloudy',


Weather_Condition
Fair                    16652
MostlyCloudy             7501
PartlyCloudy             6312
Cloudy                   3851
LightSnow                3449
FairWindy                 699
LightRain                 593
MostlyCloudyWindy         544
Snow                      496
Fog                       381
PartlyCloudyWindy         323
LightDrizzle              216
LightSnowWindy            182
Haze                      165
ShowersintheVicinity      157
ThunderintheVicinity      144
Smoke                     137
LightRainwithThunder      128
Rain                      123
HeavySnow                 121
WintryMix                 115
TStorm                    104
Thunder                    80
CloudyWindy                12
Windy                       8
Drizzle                     1
Name: count, dtype: int64

In [64]:
# Preprocess Weather_Condition

# Preprocess "Weather_Condition" column (one-hot encoding)
weather_encoder = OneHotEncoder(sparse_output=False)
weather_encoded = weather_encoder.fit_transform(df[['Weather_Condition']])
weather_columns = weather_encoder.get_feature_names_out(['Weather_Condition'])
df_weather_encoded = pd.DataFrame(weather_encoded, columns=weather_columns)
df_weather_encoded.head()


,Weather_Condition_Blowing Dust,Weather_Condition_Blowing Dust / Windy,Weather_Condition_Blowing Snow,Weather_Condition_Blowing Snow / Windy,Weather_Condition_Clear,Weather_Condition_Cloudy,Weather_Condition_Cloudy / Windy,Weather_Condition_Drizzle,Weather_Condition_Drizzle and Fog,Weather_Condition_Fair,...,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
new_list = []

for column in df_drop.columns:
    if df_drop[column].dtypes == "bool":
        new_list.append(column)
new_list

['Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop']

In [66]:
# Encoding the Bump column using a custom function

def encode_Bool(column):
    """
    This function encodes by setting yes as 1 and no as 0.
    """
    if column == "True":
        return 1
    else:
        return 0

# Call the encode_Bump function 

for column in new_list:
    df_drop[column] = df_drop[column].apply(encode_Bool)

# Review the DataFrame 
df_drop

/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_91011/1897407992.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop[column] = df_drop[column].apply(encode_Bool)


,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11,25,10,NE,8,0,PartlyCloudy,...,0,0,0,0,0,0,Day,Day,Day,Day
512525,1,86,86,13,25,10,NW,7,0,Fair,...,0,0,0,0,0,0,Day,Day,Day,Day
512588,2,71,71,26,25,10,SW,13,0,Fair,...,0,0,0,0,0,0,Day,Day,Day,Day
512593,1,78,78,21,25,10,SW,12,0,Fair,...,0,0,0,0,0,0,Day,Day,Day,Day
512594,3,86,86,13,25,10,NW,7,0,Fair,...,0,0,0,0,0,0,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14,25,10,EAST,6,0,MostlyCloudy,...,0,0,0,0,0,0,Day,Day,Day,Day
7728172,2,86,86,14,25,10,EAST,6,0,MostlyCloudy,...,0,0,0,0,0,0,Day,Day,Day,Day
7728181,3,80,80,27,25,10,SOUTH,8,0,Cloudy,...,0,0,0,0,0,0,Day,Day,Day,Day
7728192,4,76,76,36,26,10,NE,10,0,PartlyCloudy,...,0,0,0,0,0,0,Night,Night,Day,Day


In [67]:
Day_list = []
    
for column in df_drop.columns:
    if df_drop[column].dtypes == "object":
        Day_list.append(column)
Day_list

['Wind_Direction',
 'Weather_Condition',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight']

In [68]:
# Encoding the Bump column using a custom function

def encode_Day(column):
    """
    This function encodes by setting yes as 1 and no as 0.
    """
    if column == "Day":
        return 1
    else:
        return 0

# Call the encode_Bump function 
for column in Day_list:
    df_drop[column] = df_drop[column].apply(encode_Day)

# Review the DataFrame 

df_drop

/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_91011/678044661.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop[column] = df_drop[column].apply(encode_Day)


,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
512415,3,93,93,11,25,10,0,8,0,0,...,0,0,0,0,0,0,1,1,1,1
512525,1,86,86,13,25,10,0,7,0,0,...,0,0,0,0,0,0,1,1,1,1
512588,2,71,71,26,25,10,0,13,0,0,...,0,0,0,0,0,0,1,1,1,1
512593,1,78,78,21,25,10,0,12,0,0,...,0,0,0,0,0,0,1,1,1,1
512594,3,86,86,13,25,10,0,7,0,0,...,0,0,0,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728170,3,86,86,14,25,10,0,6,0,0,...,0,0,0,0,0,0,1,1,1,1
7728172,2,86,86,14,25,10,0,6,0,0,...,0,0,0,0,0,0,1,1,1,1
7728181,3,80,80,27,25,10,0,8,0,0,...,0,0,0,0,0,0,1,1,1,1
7728192,4,76,76,36,26,10,0,10,0,0,...,0,0,0,0,0,0,0,0,1,1


In [69]:
df_drop = pd.concat([ 
    df_drop,
    df_wind_encoded,
    df_weather_encoded
    ], axis=1)

df_drop = df_drop.drop(columns = ['Wind_Direction', 'Weather_Condition'])
df_drop.fillna(0, inplace=True)
df_drop

,Severity,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,...,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
512415,3.0,93.0,93.0,11.0,25.0,10.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512525,1.0,86.0,86.0,13.0,25.0,10.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512588,2.0,71.0,71.0,26.0,25.0,10.0,13.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512593,1.0,78.0,78.0,21.0,25.0,10.0,12.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512594,3.0,86.0,86.0,13.0,25.0,10.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
X = df_drop.drop(columns=['Severity'])
X.head()


,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,Bump,Crossing,...,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
512415,93.0,93.0,11.0,25.0,10.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512525,86.0,86.0,13.0,25.0,10.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512588,71.0,71.0,26.0,25.0,10.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512593,78.0,78.0,21.0,25.0,10.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512594,86.0,86.0,13.0,25.0,10.0,7.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
y = df_drop['Severity']
y_counts = y.value_counts().sort_values(ascending=False)
y_counts


Severity
0.0    90885
2.0    25120
3.0    13322
4.0     3335
1.0      717
Name: count, dtype: int64

In [72]:
#Encode Severity
le_S = LabelEncoder()
df_drop['Severity'] = le_S.fit_transform(df_drop['Severity'])
df_drop['Severity'].value_counts()

Severity
0    90885
2    25120
3    13322
4     3335
1      717
Name: count, dtype: int64

In [73]:
# df_drop['Severity'] = np.where(df_drop['Severity'].isin([0,1]),0,1)
# df_drop['Severity'].value_counts()

In [74]:
df_drop['Severity'] = np.where(df_drop['Severity'].isin([0,1]),0,np.where(df_drop['Severity'].isin([3,4]),2,1))

In [75]:
# Picked new X parameters.
X = df_drop[['Visibility(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Wind_Speed(mph)', 'Weather_Condition_Clear']]
y = df_drop['Severity']

In [76]:
X

,Visibility(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Wind_Speed(mph),Weather_Condition_Clear
512415,10.0,93.0,93.0,11.0,25.0,8.0,0.0
512525,10.0,86.0,86.0,13.0,25.0,7.0,0.0
512588,10.0,71.0,71.0,26.0,25.0,13.0,0.0
512593,10.0,78.0,78.0,21.0,25.0,12.0,0.0
512594,10.0,86.0,86.0,13.0,25.0,7.0,0.0
...,...,...,...,...,...,...,...
90880,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90881,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90882,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90883,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
y.value_counts()

Severity
0    91602
1    25120
2    16657
Name: count, dtype: int64

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [79]:
severity_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop=None)

severity_encoder.fit(np.array(y_train).reshape(-1,1))

severity_train = severity_encoder.transform(np.array(y_train).reshape(-1,1))
severity_test = severity_encoder.transform(np.array(y_test).reshape(-1,1))

severity_train, severity_test


(array([[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.]]),
 array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]))

In [80]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)
# Scale the training and testing data

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [81]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=20, activation="relu", input_dim=X.shape[1]))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
# with 5 categories
nn_model.add(tf.keras.layers.Dense(units=3, activation="softmax"))
nn_model.summary()

# Compile the Sequential model
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, severity_train, epochs=50)

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, severity_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

/Users/avalee/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 20)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 403 (1.57 KB)

 Trainable params: 403 (1.57 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 351us/step - accuracy: 0.8403 - loss: 0.3319
Epoch 2/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 331us/step - accuracy: 0.8714 - loss: 0.2343
Epoch 3/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 339us/step - accuracy: 0.8715 - loss: 0.2340
Epoch 4/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 328us/step - accuracy: 0.8712 - loss: 0.2335
Epoch 5/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 331us/step - accuracy: 0.8718 - loss: 0.2336
Epoch 6/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 346us/step - accuracy: 0.8722 - loss: 0.2319
Epoch 7/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 330us/step - accuracy: 0.8733 - loss: 0.2316
Epoch 8/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 341us/step - accuracy: 0.8713 - loss: 0.2338
Epoch 9/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 346us/step - accuracy: 0.8716 - loss: 0.2326
Epoch 10/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 329us/step - accuracy: 0.8712 - loss: 0.2339
Epoch 11/50
3127/3127 ━━━━━━━━━━━━━━━━━━━━ 1s 340us/step - accuracy: 0.8705 - loss: 0.2354
Epoch 12